Dieser Bot kann Fragen zur Sperre von Kredit- und Debitkarten sowie vom Online-Banking beantworten.
Es ist noch kein Chatbot, er kann nur einmalig auf Fragen antworten.

In [1]:
!pip install openai -q
!pip install langchain -q

In [2]:
MODEL = "gpt-4"

In [3]:
# Importing packages by third parties
import openai
from langchain.prompts import PromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.prompts import HumanMessagePromptTemplate
from langchain.schema.messages import SystemMessage
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
import json

In [4]:
# Importing packages created by us
import constants.constants as constants
import prompts.prompt_banking as prompt_banking
import prompts.prompt_cards as prompt_cards
import functions.functions as functions

In [5]:
question_of_user = "Wie kann ich meine Kreditkarte sperren lassen?"

In [6]:
# Instantiating the model
completion = openai.ChatCompletion.create(
    model = MODEL,
    api_key = constants.OPENAI_API_KEY,
    # For better understanding, the list of functions has been defined in functions/functions.py
    functions = functions.function_definition,
    messages = [
        {
            "role": "system",
            "content": """
            Du bist ein Chatbot, der auf der Website der 'Raiffeisen Bank Wien Niederösterreich' implementiert ist. Du beachstest folgende Regeln:
            
            (1) Du beantwortest NUR Fragen zur Sperrung von Debit- oder Kreditkarten oder zur Sperrung des Online-Bankings.
            (2) Du beantwortest NUR Fragen, wenn du dir absolut sicher bist.
            (3) Um die Fragen zu beantworten, rufst du die entsprechenden Funktionen auf. Die Fragen, die du an die Funktionen weitergibst, müssen vollständig ausformuliert sein.
            """
        },
        {
            "role":"user",
            "content": question_of_user
        
        }
        
    ],
    # randomness as low as possible
    temperature = 0
)

In [7]:
completion

<OpenAIObject chat.completion id=chatcmpl-8EImFGiCTwaZgl9Dd3P2UbNaPuo4Q at 0x7f1bc4f36020> JSON: {
  "id": "chatcmpl-8EImFGiCTwaZgl9Dd3P2UbNaPuo4Q",
  "object": "chat.completion",
  "created": 1698419463,
  "model": "gpt-4-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": null,
        "function_call": {
          "name": "debit_card_information",
          "arguments": "{\n  \"question\": \"Wie kann ich meine Kreditkarte sperren lassen?\"\n}"
        }
      },
      "finish_reason": "function_call"
    }
  ],
  "usage": {
    "prompt_tokens": 294,
    "completion_tokens": 27,
    "total_tokens": 321
  }
}

Es wird ein OpenAIObject zurückgegeben. Auf die oben gestellte reagiert es, indem es die folgende Funktion aufrufen will mit folgendem Inhalt aufrufen will:

In [8]:
print(question_of_user)
#print(completion["choices"][0]["message"])

print("Funktion:")
print(completion["choices"][0]["message"]["function_call"])

Wie kann ich meine Kreditkarte sperren lassen?
Funktion:
{
  "name": "debit_card_information",
  "arguments": "{\n  \"question\": \"Wie kann ich meine Kreditkarte sperren lassen?\"\n}"
}


### Definition of functions and LLM used by functions
Concerning the blocking of debit credit cards

In [9]:
llm = ChatOpenAI(openai_api_key = constants.OPENAI_API_KEY)

In [10]:
def debit_card_information(question):
    """Beantwortet Fragen zu dem Thema: Sperren von Bankomat-, Debit- und Kreditkarten."""
    chat_template = ChatPromptTemplate.from_messages(
        [
            SystemMessage(
                content=(prompt_cards.prompt_cards)
            ),
            HumanMessagePromptTemplate.from_template("Die Frage des Nutzers lautet:\n{text}"),
        ]
    )
    return llm(chat_template.format_messages(text=question))


Concerning the blocking of online banking

In [11]:
def banking_information(question):
    """Beantwortet Fragen zu dem Thema: Sperren des Online-Bankings."""
    chat_template = ChatPromptTemplate.from_messages(
        [
            SystemMessage(
                content=(prompt_banking.prompt_banking)
            ),
            HumanMessagePromptTemplate.from_template("Die Frage des Nutzers lautet:\n{text}"),
        ]
    )
    return llm(chat_template.format_messages(text=question))

In [12]:
completion = completion["choices"][0]["message"]

In [13]:
if completion.get("function_call"):
    function_name = eval(completion["function_call"]["name"])
    question = json.loads(completion["function_call"]["arguments"]).get("question")
    
    function_response = function_name(question)

In [14]:
print(function_name.__name__)
print(question)
print(function_response.content)

debit_card_information
Wie kann ich meine Kreditkarte sperren lassen?
Du kannst deine Kreditkarte sperren lassen:
(1) Über die Sperrhotline unter +43 599 320 32.
(2) Über die ELBA-APP: Logge dich ein, wähle dein Konto, gehe zu "Debitkarte verwalten", wähle die Karte aus und klicke auf "Debitkarte sperren". Gib einen Sperrgrund ein und bestätige mit dem PushTAN.
(3) Über das Online Banking "MEIN ELBA": Logge dich ein, gehe zu "Karten", wähle die Karte aus, gehe zu "Kartenverwaltung" und klicke auf "Karte sperren". Wähle "Debitkarte sperren", gib einen Sperrgrund ein und bestätige mit dem PushTAN.


In [17]:
"""
completion2 = openai.ChatCompletion.create(
    model = MODEL,
    api_key = constants.OPENAI_API_KEY,
    messages = [
        {
            "role": "system",
            "content": "Du bist ein Chatbot, der auf der Website einer Bank implementiert ist. Du beantwortest NUR Fragen zur Sperrung von Debit- oder Kreditkarten oder zur Sperrung des Online-Bankings. Du beantwortest NUR Fragen, wenn du dir absolut sicher bist. Um die Fragen zu beantworten, rufst du die entsprechenden Funktionen auf. Die Fragen, die du an die Funktionen weitergibst, müssen vollständig ausformuliert sein."
        },
        {
            "role":"user",
            "content": question_of_user
        
        },
        {
            "role":"function",
            "name": function_name.__name__,
            "content":function_response.content
        }
        
    ],
    temperature = 0

)
print(completion2)
"""

'\ncompletion2 = openai.ChatCompletion.create(\n    model = MODEL,\n    api_key = constants.OPENAI_API_KEY,\n    messages = [\n        {\n            "role": "system",\n            "content": "Du bist ein Chatbot, der auf der Website einer Bank implementiert ist. Du beantwortest NUR Fragen zur Sperrung von Debit- oder Kreditkarten oder zur Sperrung des Online-Bankings. Du beantwortest NUR Fragen, wenn du dir absolut sicher bist. Um die Fragen zu beantworten, rufst du die entsprechenden Funktionen auf. Die Fragen, die du an die Funktionen weitergibst, müssen vollständig ausformuliert sein."\n        },\n        {\n            "role":"user",\n            "content": question_of_user\n        \n        },\n        {\n            "role":"function",\n            "name": function_name.__name__,\n            "content":function_response.content\n        }\n        \n    ],\n    temperature = 0\n\n)\nprint(completion2)\n'